# Gather Bachelor students data

The purpose of this script is only to collect the data of Bachelor students, group and clean it all, serialize it in order to be used by other scripts analyzing the dataset.

## Assignment

1) Obtain all the data for the Bachelor students, starting from 2007. Keep only the students for which you have an entry for both Bachelor semestre 1 and Bachelor semestre 6.

In [1]:
import requests as r
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
ISA_url = 'http://isa.epfl.ch/imoniteur_ISAP/!gedpublicreports'
ISA_html = ISA_url + '.html'
ISA_filter = ISA_url + '.filter'
ww_i_reportModel = '133685247'  # base reference from Postman
ww_i_reportModelXsl = '133685271'  # for XLS files from Postman

### Find the parameters values

In order to find all the existing parameter names and possible values, we make a request on the form page and then parse it

In [3]:
form_html = r.get(ISA_filter, params={'ww_i_reportModel': ww_i_reportModel})
form_bs = BeautifulSoup(form_html.content, "html.parser")

We parse the HTML file looking for "select" tags which correspond to the different menus, and for each of them we look at the "option" tags which are the entries. We save our results in a dictionnary

In [4]:
params = []

for select in form_bs.find_all('select'):
    param = select['name']
    
    for option in select.find_all('option'):
        name = option.string
        
        if name:
            value = option['value']
            params.append(pd.Series([param, name, value]))

params = pd.DataFrame(params)
params.columns = ['Parameter', 'Name', 'Value']
params.set_index(['Parameter', 'Name'], inplace=True)
params

Value
Parameter           Name                                                 
ww_x_UNITE_ACAD     Architecture                                   942293
                    Chimie et génie chimique                       246696
                    Cours de mathématiques spéciales               943282
                    EME (EPFL Middle East)                      637841336
                    Génie civil                                    942623
                    Génie mécanique                                944263
                    Génie électrique et électronique               943936
                    Humanités digitales                        2054839157
                    Informatique                                   249847
                    Ingénierie financière                       120623110
                    Management de la technologie                   946882
                    Mathématiques                                  944590
                    Microtechnique                                 945244
                    Physique                                       945571
                    Science et génie des matériaux                 944917
                    Sciences et ingénierie de l'environnement      942953
                    Sciences et technologies du vivant             945901
                    Section FCUE                               1574548993
                    Systèmes de communication                      946228
ww_x_PERIODE_ACAD   2016-2017                                   355925344
                    2015-2016                                   213638028
                    2014-2015                                   213637922
                    2013-2014                                   213637754
                    2012-2013                                   123456101
                    2011-2012                                   123455150
                    2010-2011                                    39486325
                    2009-2010                                      978195
                    2008-2009                                      978187
                    2007-2008                                      978181
ww_x_PERIODE_PEDAGO Bachelor semestre 1                            249108
                    Bachelor semestre 2                            249114
                    Bachelor semestre 3                            942155
                    Bachelor semestre 4                            942163
                    Bachelor semestre 5                            942120
                    Bachelor semestre 5b                          2226768
                    Bachelor semestre 6                            942175
                    Bachelor semestre 6b                          2226785
                    Master semestre 1                             2230106
                    Master semestre 2                              942192
                    Master semestre 3                             2230128
                    Master semestre 4                             2230140
                    Mineur semestre 1                             2335667
                    Mineur semestre 2                             2335676
                    Mise à niveau                              2063602308
                    Projet Master automne                          249127
                    Projet Master printemps                       3781783
                    Semestre automne                               953159
                    Semestre printemps                            2754553
                    Stage automne 3ème année                       953137
                    Stage automne 4ème année                      2226616
                    Stage printemps 3ème année                     983606
                    Stage printemps 4ème année                    2226626
                    Stage printemps master                        222713

### Gather the data

In [5]:
def listOf(unite, year, semester):
    # retrieve gps reference using main form query for table access
    gps_html = r.get(ISA_filter, params={
        'ww_b_list': 1,
        'ww_i_reportmodel': ww_i_reportModel,
        'ww_i_reportModelXsl': ww_i_reportModelXsl,
        'ww_x_UNITE_ACAD': unite,
        'ww_x_PERIODE_ACAD': year,
        'ww_x_PERIODE_PEDAGO': semester
    })
    gps_bs = BeautifulSoup(gps_html.content, "html.parser")
    link = gps_bs.find_all('a')
    if len(link) != 2:
        return None
    
    # isolate second link (first one is always all) and rip off useless part
    gps = int(link[1]['onclick'][21:-16])
    table_html = r.get(ISA_html, params={
        'ww_i_reportmodel': ww_i_reportModel,
        'ww_i_reportModelXsl': ww_i_reportModelXsl,
        'ww_x_UNITE_ACAD': unite,
        'ww_x_PERIODE_ACAD': year,
        'ww_x_PERIODE_PEDAGO': semester,
        'ww_x_GPS': gps
    })
    table_bs = BeautifulSoup(table_html.content, "html.parser")
    tables = pd.read_html(str(table_bs), flavor='bs4', skiprows=[0, 1], header=0, index_col=10)
    
    if len(tables) != 1:
        return None
    
    table = tables[0]
    table.drop('Nom Prénom', axis=1, inplace=True)
    table.drop([col for col in table.columns if 'Unnamed' in col], axis=1, inplace=True)
    return table

In this homework we are interesting in students from the "Informatique" section, and we want the data as excel files. Therefore we give the following parameters fixed values:

In [6]:
unite = params.loc['ww_x_UNITE_ACAD', 'Informatique']
unite

Value    249847
Name: (ww_x_UNITE_ACAD, Informatique), dtype: object

In [7]:
years = params.loc['ww_x_PERIODE_ACAD']
years

,Value
Name,
2016-2017,355925344
2015-2016,213638028
2014-2015,213637922
2013-2014,213637754
2012-2013,123456101
2011-2012,123455150
2010-2011,39486325
2009-2010,978195
2008-2009,978187


In [8]:
param_semesters = params.loc['ww_x_PERIODE_PEDAGO']

In [9]:
# do not take into account "b" semester as there are all empty (not sure of signification)
bachelor_semesters = param_semesters[['Bachelor' in idx and 'b' not in idx for idx in param_semesters.index]]
bachelor_semesters

,Value
Name,
Bachelor semestre 1,249108
Bachelor semestre 2,249114
Bachelor semestre 3,942155
Bachelor semestre 4,942163
Bachelor semestre 5,942120
Bachelor semestre 6,942175


In [10]:
def gather(unite, years, semesters):
    yearly = []
    for year, year_value in years.iterrows():
        print(year)
        swarm = []
        for semester, semester_value in semesters.iterrows():
            print(semester)
            df = listOf(unite, year_value, semester_value)
            swarm.append(df)
        yearly.append(pd.concat(swarm, keys=semesters.index))
    return pd.concat(yearly, keys=years.index)

In [11]:
bachelor = gather(unite, years, bachelor_semesters)

2016-2017
Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4
Bachelor semestre 5
Bachelor semestre 6
2015-2016
Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4
Bachelor semestre 5
Bachelor semestre 6
2014-2015
Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4
Bachelor semestre 5
Bachelor semestre 6
2013-2014
Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4
Bachelor semestre 5
Bachelor semestre 6
2012-2013
Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4
Bachelor semestre 5
Bachelor semestre 6
2011-2012
Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4
Bachelor semestre 5
Bachelor semestre 6
2010-2011
Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4
Bachelor semestre 5
Bachelor semestre 6
2009-2010
Bachelor semestre 1
Bachelor semestre 2
Bachelor semestre 3
Bachelor semestre 4


In [12]:
bachelor.head()

Civilité  Orientation Bachelor  \
2016-2017 Bachelor semestre 1 235688  Monsieur                   NaN   
                              274015  Monsieur                   NaN   
                              268410    Madame                   NaN   
                              271464  Monsieur                   NaN   
                              274518  Monsieur                   NaN   

                                      Orientation Master  Spécialisation  \
2016-2017 Bachelor semestre 1 235688                 NaN             NaN   
                              274015                 NaN             NaN   
                              268410                 NaN             NaN   
                              271464                 NaN             NaN   
                              274518                 NaN             NaN   

                                     Filière opt.  Mineur   Statut  \
2016-2017 Bachelor semestre 1 235688          NaN     NaN  Présent   
                              274015          NaN     NaN  Présent   
                              268410          NaN     NaN  Présent   
                              271464          NaN     NaN  Présent   
                              274518          NaN     NaN  Présent   

                                     Type Echange Ecole Echange  
2016-2017 Bachelor semestre 1 235688          NaN           NaN  
                              274015          NaN           NaN  
                              268410          NaN           NaN  
                              271464          NaN           NaN  
                              274518          NaN           NaN

In [13]:
bachelor.to_pickle('bachelor')

In [14]:
master_semesters = param_semesters[['Master' in idx or 'Projet' in idx for idx in param_semesters.index]]
master_semesters

,Value
Name,
Master semestre 1,2230106
Master semestre 2,942192
Master semestre 3,2230128
Master semestre 4,2230140
Projet Master automne,249127
Projet Master printemps,3781783


In [15]:
master = gather(unite, years, master_semesters)

2016-2017
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Projet Master automne
Projet Master printemps
2015-2016
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Projet Master automne
Projet Master printemps
2014-2015
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Projet Master automne
Projet Master printemps
2013-2014
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Projet Master automne
Projet Master printemps
2012-2013
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Projet Master automne
Projet Master printemps
2011-2012
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Projet Master automne
Projet Master printemps
2010-2011
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Projet Master automne
Projet Master printemps
2009-2010
Master semestre 1
Master semestre 2
Master semestre 3
Master semestre 4
Projet Master automne


In [16]:
master.head()

Civilité  Orientation Bachelor  \
2016-2017 Master semestre 1 271331  Monsieur                   NaN   
                            273101  Monsieur                   NaN   
                            267543  Monsieur                   NaN   
                            202293  Monsieur                   NaN   
                            228219  Monsieur                   NaN   

                                    Orientation Master  \
2016-2017 Master semestre 1 271331                 NaN   
                            273101                 NaN   
                            267543                 NaN   
                            202293                 NaN   
                            228219                 NaN   

                                             Spécialisation  Filière opt.  \
2016-2017 Master semestre 1 271331                      NaN           NaN   
                            273101                      NaN           NaN   
                            267543                      NaN           NaN   
                            202293                      NaN           NaN   
                            228219  Foundations of Software           NaN   

                                   Mineur   Statut Type Echange Ecole Echange  
2016-2017 Master semestre 1 271331    NaN  Présent          NaN           NaN  
                            273101    NaN  Présent          NaN           NaN  
                            267543    NaN  Présent          NaN           NaN  
                            202293    NaN  Présent          NaN           NaN  
                            228219    NaN  Présent          NaN           NaN

In [17]:
master.to_pickle('master')